In [2]:
import pandas as pd
import numpy as np 
import datetime as dt
import re
import os
import geojson
import seaborn as sns
from shapely.geometry import shape, GeometryCollection
from shapely.geometry import Polygon, MultiPolygon

import matplotlib.pyplot as plt
#import scipy.stats as stats
#from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import SGDRegressor, SGDClassifier
from  math import radians
#from sklearn.pipeline import Pipeline
#from sklearn.cluster import MiniBatchKMeans, KMeans
#from sklearn.metrics import mean_squared_error, mean_absolute_error




In [3]:
raw = pd.read_parquet(f'D:/TOPIC_RELATED VOCABULARY IELTS/Finland/aalto/cemsschool/Other Courses/programming course for DS/project//trip_data/05.parquet', engine='pyarrow')


In [4]:
raw.describe()

,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
count,289480.000000,289480.000000,289480.000000,289480.000000,289480.000000,289480.000000,289480.000000,289480.000000,289480.000000,289480.000000,289480.000000,289480.000000,289480.000000,289480.000000
mean,1.314208,2.731006,1.043067,157.154066,151.584503,1.423231,12.650259,1.148115,0.491736,1.559227,0.160921,0.296522,17.478461,2.037726
std,0.984035,20.695962,0.969507,70.259291,74.551948,0.554263,798.341340,1.287057,0.081862,2.664146,1.249215,0.044436,798.479815,0.993512
min,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,-240.000000,-4.500000,-0.500000,-11.060000,-13.750000,-0.300000,-244.300000,-2.500000
25%,1.000000,0.940000,1.000000,100.000000,79.000000,1.000000,5.500000,0.000000,0.500000,0.000000,0.000000,0.300000,9.800000,2.500000
50%,1.000000,1.700000,1.000000,151.000000,147.000000,1.000000,8.000000,0.500000,0.500000,1.000000,0.000000,0.300000,12.800000,2.500000
75%,1.000000,3.100000,1.000000,234.000000,233.000000,2.000000,12.500000,2.500000,0.500000,2.450000,0.000000,0.300000,17.880000,2.500000
max,9.000000,10973.550000,99.000000,265.000000,265.000000,4.000000,429496.720000,65.530000,3.300000,442.180000,200.000000,0.300000,429562.250000,2.500000


In [5]:
raw['duration']=(raw['tpep_dropoff_datetime']-raw['tpep_pickup_datetime']).dt.total_seconds()

In [6]:
raw.isnull().any()

tpep_pickup_datetime     False
tpep_dropoff_datetime    False
passenger_count          False
trip_distance            False
RatecodeID               False
PULocationID             False
DOLocationID             False
payment_type             False
fare_amount              False
extra                    False
mta_tax                  False
tip_amount               False
tolls_amount             False
improvement_surcharge    False
total_amount             False
congestion_surcharge     False
duration                 False
dtype: bool

### HANDLING MEANINGLESS DATA

In [7]:
# LOCATION CONDITIONS
idxloc=raw['PULocationID']<=263
idx1loc=raw['DOLocationID']<=263

In [8]:
lofiltered= raw[idxloc & idx1loc]

In [9]:
lofiltered.describe()

,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,duration
count,285337.000000,285337.000000,285337.000000,285337.000000,285337.000000,285337.000000,285337.000000,285337.000000,285337.000000,285337.000000,285337.000000,285337.000000,285337.000000,285337.000000,285337.000000
mean,1.314964,2.699797,1.031310,156.091530,150.305050,1.421365,12.489593,1.154179,0.492520,1.553482,0.147832,0.296690,17.317121,2.055214,710.989483
std,0.986025,20.802041,0.806508,69.787534,73.984655,0.553233,804.093463,1.288279,0.079046,2.591901,1.161337,0.043652,804.229653,0.979458,3070.823281
min,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,-99.560000,-4.500000,-0.500000,-11.060000,-6.120000,-0.300000,-100.360000,-2.500000,0.000000
25%,1.000000,0.950000,1.000000,100.000000,76.000000,1.000000,5.500000,0.000000,0.500000,0.000000,0.000000,0.300000,9.800000,2.500000,282.000000
50%,1.000000,1.700000,1.000000,146.000000,145.000000,1.000000,8.000000,0.500000,0.500000,1.000000,0.000000,0.300000,12.800000,2.500000,465.000000
75%,1.000000,3.100000,1.000000,234.000000,233.000000,2.000000,12.500000,2.500000,0.500000,2.450000,0.000000,0.300000,17.800000,2.500000,756.000000
max,8.000000,10973.550000,99.000000,263.000000,263.000000,4.000000,429496.720000,65.530000,3.300000,442.180000,200.000000,0.300000,429562.250000,2.500000,86332.000000


In [10]:
#DEALING WITH FEATURES THAT ARE ONLY MEANINGFUL WHEN IT IS >=0
    #APPROACH1
c03=lofiltered['extra']>=0
c04=lofiltered['mta_tax']>=0
c05=lofiltered['tip_amount']>=0
c06=lofiltered['tolls_amount']>=0
c07=lofiltered['improvement_surcharge']>=0
c08=lofiltered['total_amount']>=0
c09=lofiltered['congestion_surcharge']>=0
lonilfiltered0= lofiltered[ c03 & c04 & c05 & c06 & c07 & c08 & c09]
    #APPROACH 2 
#lonilfiltered=np.abs(lofiltered.iloc[:,2::])
#lonilfiltered= pd.concat((lofiltered.iloc[:,0:2],lonilfiltered),axis=1)



In [11]:
#DEALING WITH FEATURES THAT ARE ONLY MEANINGFUL WHEN IT IS >0
    #APPROACH 1 
#DURATIOIN CONDITION
c01=  lonilfiltered0['duration']>0
# PASSENGER COUNT CONDITION 
c02= lonilfiltered0['passenger_count']>0

lonilfiltered= lonilfiltered0[c01 & c02]

In [12]:
# DEALING WITH FEATURES trip distance ==0  => 0.1
lonilfiltered['trip_distance']= lonilfiltered['trip_distance'].replace({0:0.1})


<ipython-input-12-54d16e60fcbe>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lonilfiltered['trip_distance']= lonilfiltered['trip_distance'].replace({0:0.1})


In [13]:
lonilfiltered


,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,duration
0,2020-05-01 00:02:28,2020-05-01 00:18:07,1.0,0.1,1.0,234,256,1.0,12.2,3.0,0.5,2.40,0.0,0.3,18.40,2.5,939.0
3,2020-05-01 00:07:10,2020-05-01 00:12:46,1.0,1.7,1.0,107,229,2.0,7.0,3.0,0.5,0.00,0.0,0.3,10.80,2.5,336.0
6,2020-05-01 00:09:46,2020-05-01 00:27:44,1.0,2.9,1.0,79,237,2.0,12.0,3.0,0.5,0.00,0.0,0.3,15.80,2.5,1078.0
7,2020-05-01 00:22:48,2020-05-01 00:28:58,1.0,1.5,1.0,140,263,1.0,7.0,3.0,0.5,2.15,0.0,0.3,12.95,2.5,370.0
8,2020-05-01 00:57:59,2020-05-01 01:07:48,1.0,2.0,1.0,140,230,2.0,9.0,3.0,0.5,0.00,0.0,0.3,12.80,2.5,589.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289475,2020-05-31 23:26:51,2020-05-31 23:41:12,1.0,6.1,1.0,79,75,1.0,19.0,3.0,0.5,4.55,0.0,0.3,27.35,2.5,861.0
289476,2020-05-31 23:45:06,2020-05-31 23:53:56,1.0,2.6,1.0,75,168,1.0,10.0,0.5,0.5,2.00,0.0,0.3,13.30,0.0,530.0
289477,2020-05-31 23:04:57,2020-05-31 23:11:16,1.0,1.0,1.0,41,74,2.0,6.5,0.5,0.5,0.00,0.0,0.3,7.80,0.0,379.0
289478,2020-05-31 23:20:00,2020-05-31 23:29:40,1.0,3.2,1.0,263,137,2.0,11.5,3.0,0.5,0.00,0.0,0.3,15.30,2.5,580.0


In [14]:
sum(lonilfiltered['trip_distance']==0)

0

In [15]:
# Speed conditions
sc1= (lonilfiltered['trip_distance']/lonilfiltered['duration'])<=0.009 # < 34MPH
sc2= (lonilfiltered['trip_distance']/lonilfiltered['duration'])>0.00097 # >3.5MPH
lonilspfiltered= lonilfiltered[sc1 & sc2]

In [16]:
lonilspfiltered

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,duration
3,2020-05-01 00:07:10,2020-05-01 00:12:46,1.0,1.7,1.0,107,229,2.0,7.0,3.0,0.5,0.00,0.0,0.3,10.80,2.5,336.0
6,2020-05-01 00:09:46,2020-05-01 00:27:44,1.0,2.9,1.0,79,237,2.0,12.0,3.0,0.5,0.00,0.0,0.3,15.80,2.5,1078.0
7,2020-05-01 00:22:48,2020-05-01 00:28:58,1.0,1.5,1.0,140,263,1.0,7.0,3.0,0.5,2.15,0.0,0.3,12.95,2.5,370.0
8,2020-05-01 00:57:59,2020-05-01 01:07:48,1.0,2.0,1.0,140,230,2.0,9.0,3.0,0.5,0.00,0.0,0.3,12.80,2.5,589.0
9,2020-05-01 00:10:53,2020-05-01 00:14:17,1.0,0.5,1.0,239,239,2.0,4.5,3.0,0.5,0.00,0.0,0.3,8.30,2.5,204.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289475,2020-05-31 23:26:51,2020-05-31 23:41:12,1.0,6.1,1.0,79,75,1.0,19.0,3.0,0.5,4.55,0.0,0.3,27.35,2.5,861.0
289476,2020-05-31 23:45:06,2020-05-31 23:53:56,1.0,2.6,1.0,75,168,1.0,10.0,0.5,0.5,2.00,0.0,0.3,13.30,0.0,530.0
289477,2020-05-31 23:04:57,2020-05-31 23:11:16,1.0,1.0,1.0,41,74,2.0,6.5,0.5,0.5,0.00,0.0,0.3,7.80,0.0,379.0
289478,2020-05-31 23:20:00,2020-05-31 23:29:40,1.0,3.2,1.0,263,137,2.0,11.5,3.0,0.5,0.00,0.0,0.3,15.30,2.5,580.0


In [17]:
# FARE AMOUNT CONDITIONS

fc1= lonilspfiltered['fare_amount']<lonilspfiltered['trip_distance']*10
fc2= lonilspfiltered['fare_amount']>lonilspfiltered['trip_distance']*2.5
lonilspffiltered= lonilspfiltered[fc1 & fc2]

In [18]:
lonilspffiltered

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,duration
3,2020-05-01 00:07:10,2020-05-01 00:12:46,1.0,1.7,1.0,107,229,2.0,7.0,3.0,0.5,0.00,0.0,0.3,10.80,2.5,336.0
6,2020-05-01 00:09:46,2020-05-01 00:27:44,1.0,2.9,1.0,79,237,2.0,12.0,3.0,0.5,0.00,0.0,0.3,15.80,2.5,1078.0
7,2020-05-01 00:22:48,2020-05-01 00:28:58,1.0,1.5,1.0,140,263,1.0,7.0,3.0,0.5,2.15,0.0,0.3,12.95,2.5,370.0
8,2020-05-01 00:57:59,2020-05-01 01:07:48,1.0,2.0,1.0,140,230,2.0,9.0,3.0,0.5,0.00,0.0,0.3,12.80,2.5,589.0
9,2020-05-01 00:10:53,2020-05-01 00:14:17,1.0,0.5,1.0,239,239,2.0,4.5,3.0,0.5,0.00,0.0,0.3,8.30,2.5,204.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289475,2020-05-31 23:26:51,2020-05-31 23:41:12,1.0,6.1,1.0,79,75,1.0,19.0,3.0,0.5,4.55,0.0,0.3,27.35,2.5,861.0
289476,2020-05-31 23:45:06,2020-05-31 23:53:56,1.0,2.6,1.0,75,168,1.0,10.0,0.5,0.5,2.00,0.0,0.3,13.30,0.0,530.0
289477,2020-05-31 23:04:57,2020-05-31 23:11:16,1.0,1.0,1.0,41,74,2.0,6.5,0.5,0.5,0.00,0.0,0.3,7.80,0.0,379.0
289478,2020-05-31 23:20:00,2020-05-31 23:29:40,1.0,3.2,1.0,263,137,2.0,11.5,3.0,0.5,0.00,0.0,0.3,15.30,2.5,580.0


In [19]:
raw= lonilspffiltered

In [20]:
raw

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,duration
3,2020-05-01 00:07:10,2020-05-01 00:12:46,1.0,1.7,1.0,107,229,2.0,7.0,3.0,0.5,0.00,0.0,0.3,10.80,2.5,336.0
6,2020-05-01 00:09:46,2020-05-01 00:27:44,1.0,2.9,1.0,79,237,2.0,12.0,3.0,0.5,0.00,0.0,0.3,15.80,2.5,1078.0
7,2020-05-01 00:22:48,2020-05-01 00:28:58,1.0,1.5,1.0,140,263,1.0,7.0,3.0,0.5,2.15,0.0,0.3,12.95,2.5,370.0
8,2020-05-01 00:57:59,2020-05-01 01:07:48,1.0,2.0,1.0,140,230,2.0,9.0,3.0,0.5,0.00,0.0,0.3,12.80,2.5,589.0
9,2020-05-01 00:10:53,2020-05-01 00:14:17,1.0,0.5,1.0,239,239,2.0,4.5,3.0,0.5,0.00,0.0,0.3,8.30,2.5,204.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289475,2020-05-31 23:26:51,2020-05-31 23:41:12,1.0,6.1,1.0,79,75,1.0,19.0,3.0,0.5,4.55,0.0,0.3,27.35,2.5,861.0
289476,2020-05-31 23:45:06,2020-05-31 23:53:56,1.0,2.6,1.0,75,168,1.0,10.0,0.5,0.5,2.00,0.0,0.3,13.30,0.0,530.0
289477,2020-05-31 23:04:57,2020-05-31 23:11:16,1.0,1.0,1.0,41,74,2.0,6.5,0.5,0.5,0.00,0.0,0.3,7.80,0.0,379.0
289478,2020-05-31 23:20:00,2020-05-31 23:29:40,1.0,3.2,1.0,263,137,2.0,11.5,3.0,0.5,0.00,0.0,0.3,15.30,2.5,580.0


### AUGMENTED DATASET AND DOING THE TRANSOFMATION FOR PERIODICITY FEATURES

In [21]:
# AUGMENTED DATASET
raw['start_month']=raw['tpep_pickup_datetime'].dt.month_name()
raw['end_month']=raw['tpep_dropoff_datetime'].dt.month_name()
months =['January','February','March', 'April','May','June','July','August','September','October','November','December']
dmonths= dict(zip(months,np.arange(1,13,1)))
raw['start_month_num']=raw['start_month'].map(dmonths)
raw['end_month_num']=raw['end_month'].map(dmonths)

raw['start_day']=raw['tpep_pickup_datetime'].dt.dayofweek
raw['end_day']=raw['tpep_dropoff_datetime'].dt.dayofweek
dday= dict(zip(np.arange(0,7,1),[0,0,0,0,0,1,1]))
raw['weekend_starttrip']=raw['start_day'].map(dday)
raw['weekend_endtrip']=raw['end_day'].map(dday)

raw['start_hour']=raw['tpep_pickup_datetime'].dt.hour
raw['end_hour']=raw['tpep_dropoff_datetime'].dt.hour







<ipython-input-21-2c4379d363cf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['start_month']=raw['tpep_pickup_datetime'].dt.month_name()
<ipython-input-21-2c4379d363cf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['end_month']=raw['tpep_dropoff_datetime'].dt.month_name()
<ipython-input-21-2c4379d363cf>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

In [22]:
#SIN COS TRANSFORM FOR MONTH DAY HOUR
raw['start_hour_sin']= np.sin(raw['start_hour']*(2.*np.pi/24)) # coordinate values on Oy
raw['start_hour_cos']= np.cos(raw['start_hour']*(2.*np.pi/24)) #coordinate values on Ox
raw['end_hour_sin']= np.sin(raw['end_hour']*(2.*np.pi/24)) # coordinate values on Oy
raw['end_hour_cos']= np.cos(raw['end_hour']*(2.*np.pi/24)) #coordinate values on Ox

raw['start_day_sin']= np.sin(raw['start_day']*(2.*np.pi/7))
raw['start_day_cos']= np.cos(raw['start_day']*(2.*np.pi/7))
raw['end_day_sin']= np.sin(raw['end_day']*(2.*np.pi/7))
raw['end_day_cos']= np.cos(raw['end_day']*(2.*np.pi/7))


raw['start_month_sin']= np.sin(raw['start_month_num']*(2.*np.pi/12))
raw['start_month_cos']= np.cos(raw['start_month_num']*(2.*np.pi/12))
raw['end_month_sin']= np.sin(raw['end_month_num']*(2.*np.pi/12))
raw['end_month_cos']= np.cos(raw['end_month_num']*(2.*np.pi/12))


<ipython-input-22-888254f2ea59>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['start_hour_sin']= np.sin(raw['start_hour']*(2.*np.pi/24)) # coordinate values on Oy
<ipython-input-22-888254f2ea59>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['start_hour_cos']= np.cos(raw['start_hour']*(2.*np.pi/24)) #coordinate values on Ox
<ipython-input-22-888254f2ea59>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

### HANDLING LONGITUDE LATITUDE FEATURES BASED ON LOCATIONID

In [23]:
def mapping():
    with open('D:/TOPIC_RELATED VOCABULARY IELTS/Finland/aalto/cemsschool/Other Courses/programming course for DS/project/taxi_zones.geojson') as f:
        gj = geojson.load(f)
    features = gj['features']
    return features

In [24]:
features=mapping()
coor0 = []
coor1= []


In [25]:
for i in np.arange(0,len(features)):
    if features[i]['geometry']['type']=='MultiPolygon':
        shape= features[i]['geometry']['coordinates'][0][0]
    else:   
        shape= features[i]['geometry']['coordinates'][0]
    centroid= list(Polygon(shape).centroid.coords)
    coor0.append(centroid[0][0])
    coor1.append(centroid[0][1])

In [26]:
start_pos= pd.DataFrame([coor0,coor1]).T

In [27]:
start_pos.rename(columns={0:'coor0',1:'coor1'})

,coor0,coor1
0,9.359968e+05,191376.749531
1,1.032418e+06,166735.079475
2,1.026453e+06,254265.478658
3,9.906340e+05,202959.782391
4,9.318714e+05,140681.351376
...,...,...
258,1.025106e+06,266453.414552
259,1.010212e+06,210434.891894
260,9.806395e+05,197635.173703
261,9.990266e+05,221945.372994


In [28]:
###LON LAT OF SOME SPECIFIC PLACES
JFKairport= (-73.780968, 40.641766)

NWairport=(-74.172363, 40.735657)

#MIDTOWN MANHATTAN
#UES the mark hotel 40.77541739176158, -73.96342029243233
Mark= (-73.96342029243233,40.77541739176158)
Timesquare = (-73.98604331729206,40.75837054036599)
Grand =( -73.97711770261203,40.75284316174533)
Penn= (-73.99368229963214, 40.75069857282476)
Lincoln= (-73.9839037407981, 40.773072825611635)

#West side of manhattan Bloomingdale flor de mayo 40.797800366747595, -73.9700034852982
Flor = (-73.9700034852982,40.797800366747595)

# Lower manhattan one world trade center 40.71315120718066, -74.01335094122703
Wtc=  (-74.01335094122703,40.71315120718066)

#Upper manhattan bank of america 40.82320010657351, -73.9420433120473
Boa = (-73.9420433120473, 40.82320010657351)

L= [JFKairport, NWairport, Mark,Timesquare,Grand,Penn,Lincoln,Flor,Wtc,Boa]

In [29]:
#Converting decimal lon lat to radians for each specific places mentioned above
def detora(List):
    res=[]
    for i in List:
        listc= list(map(radians,i))
        res.append(listc)
    return res


In [ ]:
res= detora(L)

In [73]:
# convert decimal lon lat to radians for PUlon and PUlat 
puloc= pd.Series(map(radians,raw['pickup_longitude']))
pulac= pd.Series(map(radians,raw['pickup_latitude']))

KeyError: 'pickup_longitude'

In [32]:
#Compute the Harvesine distance between the datapoints and each specific place listed above
def haversineindf(df):
    for i in np.arange(0,10):
        dlon= puloc-res[i][0]
        dlat= pulac-res[i][1]
        a= np.sin(dlat/2)**2 + np.cos(pulac) * cos(res[i][1]) * np.sin(dlon/2)**2
        c= 2*np.arcsin(np.sqrt(a))
        s= pd.Series(c*3956).rename(i)
        df= pd.concat((df,s),axis=1)
    return df

In [49]:
puloc

NameError: name 'puloc' is not defined

In [50]:
s1= pd.Series(np.zeros(len(puloc)))
df= haversineindf(s1)

NameError: name 'puloc' is not defined

In [51]:
df=df.iloc[:,1::]

In [52]:
df.head()

,Borough,Zone,service_zone,Unnamed: 4,Unnamed: 5
1,Queens,Jamaica Bay,Boro Zone,NaN,NaN
6,Queens,Astoria,Boro Zone,NaN,NaN
7,Queens,Astoria Park,Boro Zone,NaN,NaN
8,Queens,Auburndale,Boro Zone,NaN,NaN
9,Queens,Baisley Park,Boro Zone,NaN,NaN


In [59]:
dfscaled=dfscaled.rename(columns=dict(zip(np.arange(0,10),['tojfk','tonw', 'tomark','totimes','togrand','topenn','tolincoln','toflor','towtc','toboa'])))

NameError: name 'dfscaled' is not defined

In [60]:
#the haversine distance computed represented by a dataframe
dfscaled

NameError: name 'dfscaled' is not defined

In [61]:
Xfull= pd.concat((raw,dfscaled),axis=1)

NameError: name 'dfscaled' is not defined

#### Approach 2.2: some specific places in area having the most dropoff
(this is the initial approach using the original coordinates. You could drop this part when  the above part after being transformed to lon lat works properly. Otherwise, leave this part untouched since it could function so that you could proceed the codes in the below sections)

In [30]:
#distance to JFK
JFKdis= np.sqrt((start_pos[0]-1043400.11687483)**2+(start_pos[1]-175000.69898434)**2)
UESdis= np.sqrt((start_pos[0]-996250.38676579)**2+(start_pos[1]- 2236700.75357605)**2)
NAdis=np.sqrt((start_pos[0]-935900.82101626)**2+(start_pos[1]- 191300.74953085)**2)

Tridis= np.sqrt((start_pos[0]-982100.70138546)**2+(start_pos[1]-200700.54042726)**2)
Timdis= np.sqrt((start_pos[0]-988600.06004673)**2+(start_pos[1]-216000.13976843)**2)
UWdis= np.sqrt((start_pos[0]-993100.37686082)**2+(start_pos[1]-221000.64462577)**2)



In [31]:
dJ= dict(zip(np.arange(1,264),JFKdis))
dU= dict(zip(np.arange(1,264),UESdis))
dN= dict(zip(np.arange(1,264),NAdis))
dTr= dict(zip(np.arange(1,264),Tridis))
dTi= dict(zip(np.arange(1,264),Timdis))
dU= dict(zip(np.arange(1,264),UWdis))

In [32]:
dJSeries= raw['PULocationID'].map(dJ)
dUSeries= raw['PULocationID'].map(dU)
dNSeries=raw['PULocationID'].map(dN)
dJSeries=dJSeries.rename('JFKdis')
dUeSeries=dUSeries.rename('UESdis')
dNSeries= dNSeries.rename('NAdis')

dTrSeries= raw['PULocationID'].map(dTr)
dTiSeries= raw['PULocationID'].map(dTi)
dUSeries=raw['PULocationID'].map(dU)
dTrSeries=dTrSeries.rename('Tridis')
dTiSeries=dTiSeries.rename('Timdis')
dUwSeries=dUSeries.rename('UWdis')


In [33]:
X= pd.concat((dJSeries,dUeSeries,dNSeries, dTrSeries,dTiSeries,dUwSeries),axis=1)

In [34]:
X.describe()

,JFKdis,UESdis,NAdis,Tridis,Timdis,UWdis
count,250086.000000,250086.000000,250086.000000,250086.000000,250086.000000,250086.000000
mean,67161.724345,9863.813433,63366.505704,21393.685224,10204.421801,9863.813433
std,6914.758270,7751.527002,8981.645094,10175.458200,8095.220261,7751.527002
min,107.647573,1821.947475,122.441823,87.321246,102.883429,1821.947475
25%,64623.264905,4875.917789,57247.910439,13085.831276,5198.127000,4875.917789
50%,67788.594910,8532.896033,62650.684632,21961.476843,8278.585764,8532.896033
75%,70261.093047,12643.589157,68372.091705,27286.699324,12123.773449,12643.589157
max,131707.849207,116517.763142,129952.547662,93940.792646,109798.304133,116517.763142


In [35]:
Xfull=pd.concat((raw,X),axis=1)

### TRIMMING IRRELAVANT PARTS

In [36]:
Xfull.columns

Index(['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count',
       'trip_distance', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'total_amount',
       'congestion_surcharge', 'duration', 'start_month', 'end_month',
       'start_month_num', 'end_month_num', 'start_day', 'end_day',
       'weekend_starttrip', 'weekend_endtrip', 'start_hour', 'end_hour',
       'start_hour_sin', 'start_hour_cos', 'end_hour_sin', 'end_hour_cos',
       'start_day_sin', 'start_day_cos', 'end_day_sin', 'end_day_cos',
       'start_month_sin', 'start_month_cos', 'end_month_sin', 'end_month_cos',
       'JFKdis', 'UESdis', 'NAdis', 'Tridis', 'Timdis', 'UWdis'],
      dtype='object')

In [37]:
Xfull.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime','start_month', 'end_month',
       'start_month_num', 'end_month_num', 'start_day', 'end_day','start_hour', 'end_hour', 'PULocationID', 'DOLocationID'],axis=1, inplace=True)

In [38]:
Xuse=Xfull.loc[:,~Xfull.columns.duplicated()]

In [39]:
Xuse.columns

Index(['passenger_count', 'trip_distance', 'RatecodeID', 'payment_type',
       'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'congestion_surcharge',
       'duration', 'weekend_starttrip', 'weekend_endtrip', 'start_hour_sin',
       'start_hour_cos', 'end_hour_sin', 'end_hour_cos', 'start_day_sin',
       'start_day_cos', 'end_day_sin', 'end_day_cos', 'start_month_sin',
       'start_month_cos', 'end_month_sin', 'end_month_cos', 'JFKdis', 'UESdis',
       'NAdis', 'Tridis', 'Timdis', 'UWdis'],
      dtype='object')

### SCALING DATA

In [89]:
rs=RobustScaler()

In [90]:
rs.fit(Xuse)

RobustScaler()

In [91]:
Xscaled= pd.DataFrame(rs.transform(Xuse),index=Xuse.index,columns=Xuse.columns)


### QUEENS TASK 3

#### 3a Predicting trip distance for Queens

In [92]:
np.random.seed(0)

In [93]:

Xscaled.columns

Index(['passenger_count', 'trip_distance', 'RatecodeID', 'payment_type',
       'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'congestion_surcharge',
       'duration', 'weekend_starttrip', 'weekend_endtrip', 'start_hour_sin',
       'start_hour_cos', 'end_hour_sin', 'end_hour_cos', 'start_day_sin',
       'start_day_cos', 'end_day_sin', 'end_day_cos', 'start_month_sin',
       'start_month_cos', 'end_month_sin', 'end_month_cos', 'JFKdis', 'UESdis',
       'NAdis', 'Tridis', 'Timdis', 'UWdis'],
      dtype='object')

In [94]:
ya=Xscaled['trip_distance']

In [95]:
Xa= Xscaled.drop(['passenger_count', 'trip_distance', 'RatecodeID', 'payment_type',
       'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'congestion_surcharge',
       'duration','weekend_endtrip','end_hour_sin', 'end_hour_cos','end_day_sin','end_day_cos','end_month_sin','end_month_cos'],axis=1)


In [96]:
Xa

,weekend_starttrip,start_hour_sin,start_hour_cos,start_day_sin,start_day_cos,start_month_sin,start_month_cos,JFKdis,UESdis,NAdis,Tridis,Timdis,UWdis
3,0.0,0.189469,1.732051,-0.356896,-0.445042,0.0,0.0,-0.713499,0.703636,-0.663951,-0.869414,0.000000,0.703636
6,0.0,0.189469,1.732051,-0.356896,-0.445042,0.0,0.0,-0.925218,1.135604,-0.790567,-1.048459,0.485420,1.135604
7,0.0,0.189469,1.732051,-0.356896,-0.445042,0.0,0.0,-0.755444,-0.499791,0.350573,0.060474,0.027543,-0.499791
8,0.0,0.189469,1.732051,-0.356896,-0.445042,0.0,0.0,-0.755444,-0.499791,0.350573,0.060474,0.027543,-0.499791
9,0.0,0.189469,1.732051,-0.356896,-0.445042,0.0,0.0,0.917364,-0.470795,0.105475,0.249504,0.108897,-0.470795
...,...,...,...,...,...,...,...,...,...,...,...,...,...
289475,1.0,0.000000,1.692705,-0.643104,0.554958,0.0,0.0,-0.925218,1.135604,-0.790567,-1.048459,0.485420,1.135604
289476,1.0,0.000000,1.692705,-0.643104,0.554958,0.0,0.0,0.086969,0.018908,0.911091,0.671392,1.028375,0.018908
289477,1.0,0.000000,1.692705,-0.643104,0.554958,0.0,0.0,0.975271,0.476279,1.038500,0.937481,1.505261,0.476279
289478,1.0,0.000000,1.692705,-0.643104,0.554958,0.0,0.0,-0.282036,-0.438590,0.621338,0.374993,0.476410,-0.438590


In [97]:
Xtrain,Xtest,ytrain,ytest= train_test_split(Xa,ya,test_size=0.2)

In [98]:
# Which model, which hyperparameters to use? I do Cross-validation to check
modelhya= SGDRegressor(random_state=True)
hyperparameters= {'loss':['squared_loss','huber','squared_epsilon_insensitive'], 'alpha': np.linspace(0,0.001, 5)} 
#,'squared_epsilon_insensitive'

In [99]:
grid= GridSearchCV(modelhya, hyperparameters, cv=5, scoring='neg_root_mean_squared_error',n_jobs=-1 )

In [100]:
grid.fit(Xtrain,ytrain)

GridSearchCV(cv=5, estimator=SGDRegressor(random_state=True), n_jobs=-1,
             param_grid={'alpha': array([0.     , 0.00025, 0.0005 , 0.00075, 0.001  ]),
                         'loss': ['squared_loss', 'huber',
                                  'squared_epsilon_insensitive']},
             scoring='neg_root_mean_squared_error')

In [101]:
comparison = pd.DataFrame(np.abs(grid.cv_results_['mean_test_score']),columns=['Loss'],index=['squared0','huber0','squared_ep0','squared.25','huber.25','squared_ep.25','squared.50','huber50','squared_ep.50','squared.75','huber75','squared_ep.75','squared1','huber1','squared_ep1'])
# this score is defined by scoring. Both hypothesis maps use different loss function(squared vs squared_epsilon) to come up with the best estimate. How to tell which one performs better? Set a common scoring criteria then (working the diff between y_true and y_pred(based on best estimate of each hypothesis map. I.g: SimpleLR vs SVR)) 


In [102]:
comparison #=>squared1 best => alpha = 1, loss= squared is the best( see the results on real queens dataset). USE THESE HYPERPARAMETER FOR 3a ENTIRE DATASET

,Loss
squared0,1.216040
huber0,1.271065
squared_ep0,1.216762
squared.25,1.216038
huber.25,1.271265
squared_ep.25,1.216762
squared.50,1.216037
huber50,1.271459
squared_ep.50,1.216763
squared.75,1.216035


In [104]:
modela =SGDRegressor(alpha= 1, random_state=True)

In [105]:
modela.fit(Xtrain,ytrain)

SGDRegressor(alpha=1, random_state=True)

In [106]:
modela.score(Xtrain,ytrain)

0.08872247972138458

In [107]:
modela.score(Xtest,ytest)

0.08710572632860636

#### 3b Predicting fare_amount (parsimonious model)

In [112]:
yb= Xscaled['fare_amount']

In [113]:
Xscaled.columns

Index(['passenger_count', 'trip_distance', 'RatecodeID', 'payment_type',
       'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'congestion_surcharge',
       'duration', 'weekend_starttrip', 'weekend_endtrip', 'start_hour_sin',
       'start_hour_cos', 'end_hour_sin', 'end_hour_cos', 'start_day_sin',
       'start_day_cos', 'end_day_sin', 'end_day_cos', 'start_month_sin',
       'start_month_cos', 'end_month_sin', 'end_month_cos', 'JFKdis', 'UESdis',
       'NAdis', 'Tridis', 'Timdis', 'UWdis'],
      dtype='object')

In [114]:
Xb =Xscaled.drop(['payment_type','fare_amount',
                      'extra',
                     'mta_tax',            'tip_amount',
                'tolls_amount', 'improvement_surcharge',
                'total_amount',  'congestion_surcharge',
                ],axis=1)

In [115]:
Xb.columns

Index(['passenger_count', 'trip_distance', 'RatecodeID', 'duration',
       'weekend_starttrip', 'weekend_endtrip', 'start_hour_sin',
       'start_hour_cos', 'end_hour_sin', 'end_hour_cos', 'start_day_sin',
       'start_day_cos', 'end_day_sin', 'end_day_cos', 'start_month_sin',
       'start_month_cos', 'end_month_sin', 'end_month_cos', 'JFKdis', 'UESdis',
       'NAdis', 'Tridis', 'Timdis', 'UWdis'],
      dtype='object')

In [116]:
Xtrain,Xtest, ytrain,ytest = train_test_split(Xb,yb,test_size=0.2)

In [117]:
# Which model, which hyperparameters to use? I do the Cross-validation to check
modelhyb= SGDRegressor(random_state=True)
hyperparameters= {'loss':['squared_loss','huber','squared_epsilon_insensitive'], 'alpha': np.linspace(0,0.001, 5)} 
#,'squared_epsilon_insensitive'

In [118]:
grid= GridSearchCV(modelhyb, hyperparameters, cv=5, scoring='neg_root_mean_squared_error',n_jobs=-1 )

In [119]:
grid.fit(Xtrain,ytrain)

GridSearchCV(cv=5, estimator=SGDRegressor(random_state=True), n_jobs=-1,
             param_grid={'alpha': array([0.     , 0.00025, 0.0005 , 0.00075, 0.001  ]),
                         'loss': ['squared_loss', 'huber',
                                  'squared_epsilon_insensitive']},
             scoring='neg_root_mean_squared_error')

In [120]:
comparison = pd.DataFrame(np.abs(grid.cv_results_['mean_test_score']),columns=['Loss'],index=['squared0','huber0','squared_ep0','squared.25','huber.25','squared_ep.25','squared.50','huber50','squared_ep.50','squared.75','huber75','squared_ep.75','squared1','huber1','squared_ep1'])

# this score is defined by scoring. Both hypothesis maps use different loss function(squared vs squared_epsilon) to come up with the best estimate. How to tell which one performs better? Set a common scoring criteria then (working the diff between y_true and y_pred(based on best estimate of each hypothesis map. I.g: SimpleLR vs SVR)) 


In [121]:
comparison #=>squared1 best => alpha = .25, loss= squared_loss is the best. USE THESE HYPERPARAMETER FOR 3a ENTIRE DATASET

,Loss
squared0,8.378251
huber0,0.170956
squared_ep0,2047.671134
squared.25,7.601768
huber.25,0.170977
squared_ep.25,1842.568523
squared.50,6.897658
huber50,0.170993
squared_ep.50,1658.202629
squared.75,6.259161


In [122]:
modelb =SGDRegressor(alpha= .25, random_state=True)

In [123]:
modelb.fit(Xtrain,ytrain)

SGDRegressor(alpha=0.25, random_state=True)

In [124]:
modelb.score(Xtrain,ytrain)

0.9804424957989455

In [125]:
modelb.score(Xtest,ytest)

0.9878430993523114

In [384]:
Xscaled.corr()

,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,...,start_month_sin,start_month_cos,end_month_sin,end_month_cos,JFKdis,UESdis,NAdis,Tridis,Timdis,UWdis
passenger_count,1.000000,0.010001,-0.002966,-0.002624,0.010392,-0.163364,0.015422,0.002412,0.002688,0.004451,...,-0.001705,-0.001082,-0.000476,-0.000941,-0.002091,0.009865,0.007464,0.007923,0.014112,0.009865
trip_distance,0.010001,1.000000,0.055931,-0.012342,0.977954,-0.003467,-0.039438,0.346553,0.442942,-0.021732,...,0.000687,-0.000941,-0.014712,0.003142,-0.372150,0.409168,0.257738,0.217359,0.390136,0.409168
RatecodeID,-0.002966,0.055931,1.000000,0.003145,0.057443,-0.003604,-0.060524,0.024021,0.052469,-0.002013,...,-0.000057,-0.000057,-0.000764,0.000166,-0.044354,0.040811,0.030229,0.028145,0.041166,0.040811
payment_type,-0.002624,-0.012342,0.003145,1.000000,-0.007369,-0.033047,-0.001694,-0.542634,-0.014110,-0.011875,...,0.003699,0.003451,0.000558,0.003330,0.000398,0.050943,0.064753,0.068134,0.069915,0.050943
fare_amount,0.010392,0.977954,0.057443,-0.007369,1.000000,0.001258,-0.055967,0.349590,0.420287,-0.024548,...,0.000409,-0.000888,-0.012579,0.002624,-0.334942,0.379785,0.238026,0.202517,0.364048,0.379785
extra,-0.163364,-0.003467,-0.003604,-0.033047,0.001258,1.000000,-0.034379,0.036549,-0.012336,-0.000496,...,-0.002118,-0.002480,-0.001595,-0.001975,0.011319,-0.080023,-0.120102,-0.125098,-0.126779,-0.080023
mta_tax,0.015422,-0.039438,-0.060524,-0.001694,-0.055967,-0.034379,1.000000,-0.018659,-0.073116,0.008357,...,0.000146,0.000146,-0.000494,0.000280,0.004193,-0.007481,-0.006724,-0.007614,-0.009296,-0.007481
tip_amount,0.002412,0.346553,0.024021,-0.542634,0.349590,0.036549,-0.018659,1.000000,0.191610,0.006443,...,-0.002149,-0.002084,-0.002304,-0.001411,-0.130923,0.103609,0.039656,0.020798,0.083747,0.103609
tolls_amount,0.002688,0.442942,0.052469,-0.014110,0.420287,-0.012336,-0.073116,0.191610,1.000000,-0.005888,...,-0.000382,-0.000383,-0.009279,0.002303,-0.175615,0.198026,0.189170,0.155462,0.211100,0.198026
improvement_surcharge,0.004451,-0.021732,-0.002013,-0.011875,-0.024548,-0.000496,0.008357,0.006443,-0.005888,1.000000,...,0.000033,0.000034,-0.000113,0.000064,0.004717,-0.006326,-0.003627,-0.003017,-0.006231,-0.006326


##### 8 strongest features trip_distance and duration + 6 distance features

In [413]:
modelb8= SGDRegressor(alpha=.25,random_state=True)

In [415]:
modelb8.fit(Xtrain[['trip_distance','duration','JFKdis','UESdis','NAdis','Tridis','Timdis','UWdis']],ytrain)

SGDRegressor(alpha=0.25, random_state=True)

In [416]:
modelb8.score(Xtrain[['trip_distance','duration','JFKdis','UESdis','NAdis','Tridis','Timdis','UWdis']],ytrain)

0.9802298269203806

In [432]:
modelb8.score(Xtest[['trip_distance','duration','JFKdis','UESdis','NAdis','Tridis','Timdis','UWdis']],ytest)

NameError: name 'modelb8' is not defined

##### What about 6 features which are not really strongly linearly correalated to the fare_amount

In [426]:
modelb6= SGDRegressor(alpha=.25,random_state=True)

In [427]:
modelb6.fit(Xtrain[['JFKdis','UESdis','NAdis','Tridis','Timdis','UWdis']],ytrain)

SGDRegressor(alpha=0.25, random_state=True)

In [428]:
modelb6.score(Xtrain[['JFKdis','UESdis','NAdis','Tridis','Timdis','UWdis']],ytrain)

0.1636583740198263

In [429]:
modelb6.score(Xtest[['JFKdis','UESdis','NAdis','Tridis','Timdis','UWdis']],ytest)

0.1674989633191657

##### 2 strongest features trip_distance and duration

In [409]:
modelb2= SGDRegressor(alpha=.25,random_state=True)

In [410]:
modelb2.fit(Xtrain[['trip_distance','duration']],ytrain)

SGDRegressor(alpha=0.25, random_state=True)

In [411]:
modelb2.score(Xtrain[['trip_distance','duration']],ytrain)

0.9811036953226984

In [412]:
modelb2.score(Xtest[['trip_distance','duration']],ytest)

0.9850873243666176

##### Could the model be more parsimonious? 1 feature only. Either trip_distance or duration

In [418]:
modelb1= SGDRegressor(alpha=.25,random_state=True)

In [419]:
modelb1.fit(Xtrain[['trip_distance']],ytrain)

SGDRegressor(alpha=0.25, random_state=True)

In [420]:
modelb1.score(Xtrain[['trip_distance']],ytrain)

0.9418774728552727

In [421]:
modelb1.score(Xtest[['trip_distance']],ytest)

0.9414299598377417

In [422]:
modelbdu= SGDRegressor(alpha=.25,random_state=True)

In [423]:
modelbdu.fit(Xtrain[['duration']],ytrain)

SGDRegressor(alpha=0.25, random_state=True)

In [441]:
modelbdu.score(Xtrain[['duration']],ytrain)


0.8239626235232147

In [442]:
modelbdu.score(Xtest[['duration']],ytest)

0.8317269995519062

####### => conclusion: 2 features should be best both in terms of the performance of the model and the number of parameters

#### 3c Predict payment type

In [126]:
Xscaled.columns

Index(['passenger_count', 'trip_distance', 'RatecodeID', 'payment_type',
       'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'congestion_surcharge',
       'duration', 'weekend_starttrip', 'weekend_endtrip', 'start_hour_sin',
       'start_hour_cos', 'end_hour_sin', 'end_hour_cos', 'start_day_sin',
       'start_day_cos', 'end_day_sin', 'end_day_cos', 'start_month_sin',
       'start_month_cos', 'end_month_sin', 'end_month_cos', 'JFKdis', 'UESdis',
       'NAdis', 'Tridis', 'Timdis', 'UWdis'],
      dtype='object')

In [127]:
yc= Xscaled['payment_type']

In [128]:
Xc=Xscaled.drop(['payment_type'],axis=1)

In [129]:
Xtrain,Xtest, ytrain,ytest = train_test_split(Xc,yc,test_size=0.2)

In [130]:
# Which model, which hyperparameters to use? I do the Cross-validation to check
modelhyc= SGDClassifier(random_state=True)
hyperparameters= {'loss':['log','hinge','squared_hinge'], 'alpha': np.linspace(0.0001,0.001, 5)} 


In [131]:
ytrain

120138    1.0
30174     1.0
97245     0.0
141347    0.0
193809    1.0
         ... 
224666    0.0
183653    0.0
207615    1.0
17308     0.0
101171    1.0
Name: payment_type, Length: 200068, dtype: float64

In [132]:
grid= GridSearchCV(modelhyc, hyperparameters, cv=5, scoring='neg_root_mean_squared_error',n_jobs=-1 )

In [133]:
grid.fit(Xtrain,ytrain)

KeyboardInterrupt: 

In [140]:
comparison = pd.DataFrame(np.abs(grid.cv_results_['mean_test_score']),columns=['Loss'],index=['squared0','squared_ep0','squared.25','squared_ep.25','squared.50','squared_ep.50','squared.75','squared_ep.75','squared1','squared_ep1'])
# this score is defined by scoring. Both hypothesis maps use different loss function(squared vs squared_epsilon) to come up with the best estimate. How to tell which one performs better? Set a common scoring criteria then (working the diff between y_true and y_pred(based on best estimate of each hypothesis map. I.g: SimpleLR vs SVR)) 


AttributeError: 'GridSearchCV' object has no attribute 'cv_results_'

In [378]:
comparison #=>squared1 best => alpha = .25, loss= squared_loss is the best. USE THESE HYPERPARAMETER FOR 3a ENTIRE DATASET

,Loss
squared0,1.344923e+09
squared_ep0,5.334084e+09
squared.25,1.251926e+09
squared_ep.25,5.320682e+09
squared.50,1.291309e+09
squared_ep.50,5.305431e+09
squared.75,1.414075e+09
squared_ep.75,5.290541e+09
squared1,1.521205e+09
squared_ep1,5.277427e+09


In [135]:
modelc= SGDClassifier(alpha=.25,random_state=True)

In [136]:
modelc.fit(Xtrain,ytrain)

SGDClassifier(alpha=0.25, random_state=True)

In [137]:
modelc.score(Xtrain,ytrain)


0.7905012295819421

In [139]:
modelc.score(Xtest,ytest)

0.790015594386021

### ENTIRE DATASET TASK 3

#### 3a Predicting the trip_distance

In [356]:
modelae =SGDRegressor(alpha= 1, random_state=True) #best hyperparameter found in Queens would be used 

In [357]:
# Place it within loop through chunks
modelae.partial_fit(Xtrain,ytrain)

SGDRegressor(alpha=1, random_state=True)

In [358]:
# Place it out of the loop
modelae.score(Xtrain,ytrain)

0.1970052542776891

In [359]:
# place it out of the loop
modelae.score(Xtest,ytest)

0.1927796251879833

#### 3b Predicting the fare_amount

In [108]:
modelb =SGDRegressor(alpha= .25, random_state=True)

In [109]:
# Place it within loop through chunks
modelb.partial_fit(Xtrain,ytrain)

SGDRegressor(alpha=0.25, random_state=True)

In [110]:
#PLace it out of the loop
modelb.score(Xtrain,ytrain)

0.09338216548053413

In [111]:
#place it out of the loop
modelb.score(Xtest,ytest)

0.09273576372404524

In [384]:
Xscaled.corr()

,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,...,start_month_sin,start_month_cos,end_month_sin,end_month_cos,JFKdis,UESdis,NAdis,Tridis,Timdis,UWdis
passenger_count,1.000000,0.010001,-0.002966,-0.002624,0.010392,-0.163364,0.015422,0.002412,0.002688,0.004451,...,-0.001705,-0.001082,-0.000476,-0.000941,-0.002091,0.009865,0.007464,0.007923,0.014112,0.009865
trip_distance,0.010001,1.000000,0.055931,-0.012342,0.977954,-0.003467,-0.039438,0.346553,0.442942,-0.021732,...,0.000687,-0.000941,-0.014712,0.003142,-0.372150,0.409168,0.257738,0.217359,0.390136,0.409168
RatecodeID,-0.002966,0.055931,1.000000,0.003145,0.057443,-0.003604,-0.060524,0.024021,0.052469,-0.002013,...,-0.000057,-0.000057,-0.000764,0.000166,-0.044354,0.040811,0.030229,0.028145,0.041166,0.040811
payment_type,-0.002624,-0.012342,0.003145,1.000000,-0.007369,-0.033047,-0.001694,-0.542634,-0.014110,-0.011875,...,0.003699,0.003451,0.000558,0.003330,0.000398,0.050943,0.064753,0.068134,0.069915,0.050943
fare_amount,0.010392,0.977954,0.057443,-0.007369,1.000000,0.001258,-0.055967,0.349590,0.420287,-0.024548,...,0.000409,-0.000888,-0.012579,0.002624,-0.334942,0.379785,0.238026,0.202517,0.364048,0.379785
extra,-0.163364,-0.003467,-0.003604,-0.033047,0.001258,1.000000,-0.034379,0.036549,-0.012336,-0.000496,...,-0.002118,-0.002480,-0.001595,-0.001975,0.011319,-0.080023,-0.120102,-0.125098,-0.126779,-0.080023
mta_tax,0.015422,-0.039438,-0.060524,-0.001694,-0.055967,-0.034379,1.000000,-0.018659,-0.073116,0.008357,...,0.000146,0.000146,-0.000494,0.000280,0.004193,-0.007481,-0.006724,-0.007614,-0.009296,-0.007481
tip_amount,0.002412,0.346553,0.024021,-0.542634,0.349590,0.036549,-0.018659,1.000000,0.191610,0.006443,...,-0.002149,-0.002084,-0.002304,-0.001411,-0.130923,0.103609,0.039656,0.020798,0.083747,0.103609
tolls_amount,0.002688,0.442942,0.052469,-0.014110,0.420287,-0.012336,-0.073116,0.191610,1.000000,-0.005888,...,-0.000382,-0.000383,-0.009279,0.002303,-0.175615,0.198026,0.189170,0.155462,0.211100,0.198026
improvement_surcharge,0.004451,-0.021732,-0.002013,-0.011875,-0.024548,-0.000496,0.008357,0.006443,-0.005888,1.000000,...,0.000033,0.000034,-0.000113,0.000064,0.004717,-0.006326,-0.003627,-0.003017,-0.006231,-0.006326


##### 8 strongest features trip_distance and duration + 6 distance features (if could, create a table or a file incorporating only these 8 features from the outset. Doing that would help reduce the time it takes to go about the codes)

In [413]:
modelb8= SGDRegressor(alpha=.25,random_state=True)

In [415]:
# within loop through chunks
modelb8.partial_fit(Xtrain[['trip_distance','duration','JFKdis','UESdis','NAdis','Tridis','Timdis','UWdis']],ytrain)

SGDRegressor(alpha=0.25, random_state=True)

In [416]:
# out of loop
modelb8.score(Xtrain[['trip_distance','duration','JFKdis','UESdis','NAdis','Tridis','Timdis','UWdis']],ytrain)

0.9802298269203806

In [432]:
# out of loop
modelb8.score(Xtest[['trip_distance','duration','JFKdis','UESdis','NAdis','Tridis','Timdis','UWdis']],ytest)

NameError: name 'modelb8' is not defined

##### What about 6 features which are not really strongly linearly correalated to the fare_amount (same idea as above. Create a table or a file of only these 6 features before running model would be helpful to cut the waiting time drastically)

In [426]:
modelb6= SGDRegressor(alpha=.25,random_state=True)

In [427]:
# within loop through chunks
modelb6.partial_fit(Xtrain[['JFKdis','UESdis','NAdis','Tridis','Timdis','UWdis']],ytrain)

SGDRegressor(alpha=0.25, random_state=True)

In [428]:
#out of loop
modelb6.score(Xtrain[['JFKdis','UESdis','NAdis','Tridis','Timdis','UWdis']],ytrain)

0.1636583740198263

In [429]:
# out of loop
modelb6.score(Xtest[['JFKdis','UESdis','NAdis','Tridis','Timdis','UWdis']],ytest)

0.1674989633191657

##### 2 strongest features trip_distance and duration(same idea as above)

In [409]:
modelb2= SGDRegressor(alpha=.25,random_state=True)

In [410]:
#within loop
modelb2.partial_fit(Xtrain[['trip_distance','duration']],ytrain)

SGDRegressor(alpha=0.25, random_state=True)

In [411]:
# out of loop
modelb2.score(Xtrain[['trip_distance','duration']],ytrain)

0.9811036953226984

In [412]:
# out of loop
modelb2.score(Xtest[['trip_distance','duration']],ytest)

0.9850873243666176

##### Could the model be more parsimonious? 1 feature only. Either trip_distance or duration

In [418]:
modelb1= SGDRegressor(alpha=.25,random_state=True)

In [419]:
#within loop
modelb1.partial_fit(Xtrain[['trip_distance']],ytrain)

SGDRegressor(alpha=0.25, random_state=True)

In [420]:
# out of loop
modelb1.score(Xtrain[['trip_distance']],ytrain)

0.9418774728552727

In [421]:
# out of loop
modelb1.score(Xtest[['trip_distance']],ytest)

0.9414299598377417

In [422]:
modelbdu= SGDRegressor(alpha=.25,random_state=True)

In [423]:
#within loop
modelbdu.partial_fit(Xtrain[['duration']],ytrain)

SGDRegressor(alpha=0.25, random_state=True)

In [424]:
#out of loop
modelbdu.score(Xtrain[['duration']],ytrain)

0.8239626235232147

In [425]:
#out of loop
modelbdu.score(Xtest[['duration']],ytest)

0.8317269995519062

####### => conclusion: 2 features should be best both in terms of the performance of the model and the number of parameters

#### 3c Predict payment type

In [126]:
Xscaled.columns

Index(['passenger_count', 'trip_distance', 'RatecodeID', 'payment_type',
       'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'congestion_surcharge',
       'duration', 'weekend_starttrip', 'weekend_endtrip', 'start_hour_sin',
       'start_hour_cos', 'end_hour_sin', 'end_hour_cos', 'start_day_sin',
       'start_day_cos', 'end_day_sin', 'end_day_cos', 'start_month_sin',
       'start_month_cos', 'end_month_sin', 'end_month_cos', 'JFKdis', 'UESdis',
       'NAdis', 'Tridis', 'Timdis', 'UWdis'],
      dtype='object')

In [127]:
yc= Xscaled['payment_type']

In [128]:
Xc=Xscaled.drop(['payment_type'],axis=1)

In [129]:
Xtrain,Xtest, ytrain,ytest = train_test_split(Xc,yc,test_size=0.2)

In [135]:
modelc= SGDClassifier(alpha=.25,random_state=True)

In [136]:
#within the loop
modelc.partial_fit(Xtrain,ytrain)

SGDClassifier(alpha=0.25, random_state=True)

In [137]:
#out of loop
modelc.score(Xtrain,ytrain)


0.7905012295819421

In [139]:
#out of loop
modelc.score(Xtest,ytest)

0.790015594386021